In [1]:
import pandas as pd
import logging
import networkx as nx
import time
import json


from causal_reasoning.causal_model import CausalModel


In [ ]:
class Query:
    def __init__(self, inter_label: str, inter_value: int, target_label: str, target_value: int, query_value: float) -> None:
        self.intervention_label = inter_label
        self.intervention_value = inter_value
        self.target_label = target_label
        self.target_value = target_value
        self.query = query_value

def run_queries(nodes_set: set, model: CausalModel, empirical_probs, output_path: str):
    with open(output_path, 'w') as file:
        queries = []
        for intervention in nodes_set:
            for target in nodes_set:
                if intervention == target:
                    continue
                intervention_value = 0
                target_value = 0
                model.set_interventions([(intervention, intervention_value)])
                model.set_target((target, target_value))
                start = time.perf_counter()
                pn = model.inference_intervention_query()
                end = time.perf_counter()
                pn_time = end-start
                queries.append(Query(intervention, intervention_value, target, target_value, pn))


                intervention_value = 1
                target_value = 1
                model.set_interventions([(intervention, intervention_value)])
                model.set_target((target, target_value))
                start = time.perf_counter()
                ps = model.inference_intervention_query()
                end = time.perf_counter()
                ps_time = end-start
                queries.append(Query(intervention, intervention_value, target, target_value, ps))
                
                # file.write(f"PS = P({target}={target_value}|do({intervention}={intervention_value}))\n")
                file.write(f"PS = P({target}={1}|do({intervention}={1}))\n")
                file.write(f"{ps[0]} <= PS <= {ps[1]}\n")
                file.write(f"PS Inference time taken: {ps_time:.6f} seconds\n")
                file.write("----\n")
                file.write(f"PN = P({target}={0}|do({intervention}={0}))\n")
                file.write(f"{pn[0]} <= PN <= {pn[1]}\n")
                file.write(f"PN Inference time taken: {pn_time:.6f} seconds\n")
                file.write("----\n")

                empirical_ps = empirical_probs[intervention][1][target]
                empirical_pn = empirical_probs[intervention][0][target]
                
                if ps[0] == 'None' or ps[0] is None or ps[1] == 'None' or ps[1] is None or pn[0] == 'None' or pn[0] is None or pn[1] == 'None' or pn[1] is None: 
                    file.write(f"PNS = P({target}|do({intervention}))\n")
                    file.write(f"PNS == None\n")
                    file.write("---------------------------------------------------------\n")
                    continue

                ps_lower = empirical_ps*float(ps[0])
                ps_upper = empirical_ps*float(ps[1])
                pn_lower = empirical_pn*float(pn[0])
                pn_upper = empirical_pn*float(pn[1])

                pns = [0,0]
                pns[0] = min(ps_lower+pn_lower, ps_lower+pn_upper)
                pns[1] = max(ps_upper+pn_lower, ps_upper+pn_upper)
                file.write(f"PNS = P({target}|do({intervention}))\n")
                file.write(f"{pn[0]} <= PNS <= {pn[1]}\n")
                file.write("---------------------------------------------------------\n")


def run_pn_ps(nodes_set: set, model: CausalModel, target: str, output_path: str):
    pn_ps = {}
    for intervention in nodes_set:
        if intervention == target:
            continue
        pn_ps[intervention] = {}

        intervention_value = 0
        target_value = 0
        model.set_interventions([(intervention, intervention_value)])
        model.set_target((target, target_value))
        start = time.perf_counter()
        pn = model.inference_intervention_query()
        end = time.perf_counter()
        pn_time = end-start
        pn_ps[intervention]["PN"] = pn

        intervention_value = 1
        target_value = 1
        model.set_interventions([(intervention, intervention_value)])
        model.set_target((target, target_value))
        start = time.perf_counter()
        ps = model.inference_intervention_query()
        end = time.perf_counter()
        ps_time = end-start
        pn_ps[intervention]["PS"] = ps
            
    with open(f'{output_path}.json', 'w') as f:
        json.dump(pn_ps, f, indent=4)



## First Model

In [ ]:
edges_list_1 = [
    ('NewDeploy', 'MemoryLeak'),
    ('MemoryLeak', 'MemoryUsageHigh'), 
    ("MemoryUsageHigh", "ServiceCrash"), 
    ("ServiceCrash", "OutageIncident"),
    ("HeavyTraffic", "MemoryUsageHigh"), 
    ("HeavyTraffic", "ServiceCrash"),
    # UNOBS
    ('Unob_helper_1', 'MemoryLeak'),
    ('Unob_helper_2', 'OutageIncident'),
    ('Unob_helper_3', 'NewDeploy'),
]
latent_nodes_1 = ['HeavyTraffic', 'Unob_helper_1', 'Unob_helper_2', 'Unob_helper_3']
nodes_set_1 = set()
for tuple in edges_list_1:
    if tuple[0] not in latent_nodes_1:
        nodes_set_1.add(tuple[0])
    if tuple[1] not in latent_nodes_1:
        nodes_set_1.add(tuple[1])

edges_1 = nx.DiGraph(edges_list_1)

df_small_scale_model = pd.read_csv("small_scale_outage_incident_seed42.csv", index_col=0)

In [ ]:
empirical_probs_model_1 = {}
for intervention in nodes_set_1:
    empirical_probs_model_1[intervention] = [{},{}]
    for target in nodes_set_1:
        if target == intervention:
            continue
        empirical_probs_model_1[intervention][0][target] = df_small_scale_model[(df_small_scale_model[intervention] == 0) & (df_small_scale_model[target] == 0)].shape[0]
        empirical_probs_model_1[intervention][1][target] = df_small_scale_model[(df_small_scale_model[intervention] == 1) & (df_small_scale_model[target] == 1)].shape[0]

In [ ]:
model_1 = CausalModel(
    data=df_small_scale_model,
    edges=edges_1,
    unobservables_labels=latent_nodes_1,
)
# run_queries(nodes_set_1, model_1, empirical_probs_model_1, "output_small_scale_outage_incident_seed42.txt")
# run_pn_ps(nodes_set_1, model_1, "OutageIncident", "output_small_scale_outage_incident_seed42")

## Second Model


In [ ]:
edges_list_2 = [
    ("DB_Change", "DB_Latency"),
    ("DB_Latency", "MS-B_Latency"), 
    ("MS-B_Latency", "MS-B_Error"), 
    ("MS-B_Latency", "MS-A_Latency"),
    ("MS-B_Error", "MS-A_Error"), 
    ("MS-A_Latency", "MS-A_Threads"), 
    ("MS-A_Threads", "MS-A_Crash"), 
    ("MS-A_Error", "Outage"), 
    ("MS-A_Crash", "Outage"), 
    ("HeavyTraffic", "DB_Latency"), 
    ("HeavyTraffic", "MS-A_Latency"),
    # UNOBS
    ('Unob_helper_1', 'DB_Change'),
    ('Unob_helper_2', 'MS-B_Latency'),
    ('Unob_helper_3', 'MS-B_Error'),
    ('Unob_helper_4', 'MS-A_Error'),
    ('Unob_helper_5', 'MS-A_Threads'),
    ('Unob_helper_6', 'MS-A_Crash'),
    ('Unob_helper_7', 'Outage'),
]

latent_nodes_2 = ['HeavyTraffic', 'Unob_helper_1', 'Unob_helper_2', 'Unob_helper_3', 'Unob_helper_4', 'Unob_helper_5', 'Unob_helper_6', 'Unob_helper_7']
nodes_set_2 = set()
for tuple in edges_list_2:
    if tuple[0] not in latent_nodes_2:
        nodes_set_2.add(tuple[0])
    if tuple[1] not in latent_nodes_2:
        nodes_set_2.add(tuple[1])

edges_2 = nx.DiGraph(edges_list_2)

df_medium_scale_incident = pd.read_csv("medium_scale_outage_incident_seed42.csv", index_col=0)

In [ ]:
empirical_probs_model_2 = {}
for intervention in nodes_set_2:
    empirical_probs_model_2[intervention] = [{},{}]
    for target in nodes_set_2:
        if target == intervention:
            continue
        empirical_probs_model_2[intervention][0][target] = df_medium_scale_incident[(df_medium_scale_incident[intervention] == 0) & (df_medium_scale_incident[target] == 0)].shape[0]
        empirical_probs_model_2[intervention][1][target] = df_medium_scale_incident[(df_medium_scale_incident[intervention] == 1) & (df_medium_scale_incident[target] == 1)].shape[0]

In [ ]:
model_2 = CausalModel(
    data=df_medium_scale_incident,
    edges=edges_2,
    unobservables_labels=latent_nodes_2,
)
# run_queries(nodes_set_2, model_2, empirical_probs_model_2, "output_medium_scale_outage_incident_seed42.txt")
# run_pn_ps(nodes_set_2, model_2, "Outage", "output_medium_scale_outage_incident_seed42")

In [4]:
import dowhy
import networkx as nx
import pandas as pd


edges_list_2 = [
    ("DB_Change", "DB_Latency"),
    ("DB_Latency", "MS-B_Latency"), 
    ("MS-B_Latency", "MS-B_Error"), 
    ("MS-B_Latency", "MS-A_Latency"),
    ("MS-B_Error", "MS-A_Error"), 
    ("MS-A_Latency", "MS-A_Threads"), 
    ("MS-A_Threads", "MS-A_Crash"), 
    ("MS-A_Error", "Outage"), 
    ("MS-A_Crash", "Outage"), 
    ("HeavyTraffic", "DB_Latency"), 
    ("HeavyTraffic", "MS-A_Latency"),
    # UNOBS
    ('Unob_helper_1', 'DB_Change'),
    ('Unob_helper_2', 'MS-B_Latency'),
    ('Unob_helper_3', 'MS-B_Error'),
    ('Unob_helper_4', 'MS-A_Error'),
    ('Unob_helper_5', 'MS-A_Threads'),
    ('Unob_helper_6', 'MS-A_Crash'),
    ('Unob_helper_7', 'Outage'),
]

latent_nodes_2 = ['HeavyTraffic', 'Unob_helper_1', 'Unob_helper_2', 'Unob_helper_3', 'Unob_helper_4', 'Unob_helper_5', 'Unob_helper_6', 'Unob_helper_7']
nodes_set_2 = set()
for tuple in edges_list_2:
    if tuple[0] not in latent_nodes_2:
        nodes_set_2.add(tuple[0])
    if tuple[1] not in latent_nodes_2:
        nodes_set_2.add(tuple[1])

edges_2 = nx.DiGraph(edges_list_2)

df_medium_scale_incident = pd.read_csv("medium_scale_outage_incident_seed42.csv", index_col=0)



In [7]:
mod = dowhy.CausalModel(data=df_medium_scale_incident,graph=edges_2, treatment="DB_Latency", outcome="Outage")

/home/lawand/.local/lib/python3.10/site-packages/dowhy/causal_model.py:582: UserWarning: 7 variables are assumed unobserved because they are not in the dataset. Configure the logging level to `logging.WARNING` or higher for additional details.
  warnings.warn(


In [10]:
# Step 2: Identify
identified_estimand = mod.identify_effect(proceed_when_unidentifiable=True)
print(identified_estimand)

Estimand type: EstimandType.NONPARAMETRIC_ATE

### Estimand : 1
Estimand name: backdoor
Estimand expression:
      d                              
─────────────(E[Outage|HeavyTraffic])
d[DB_Latency]                        
Estimand assumption 1, Unconfoundedness: If U→{DB_Latency} and U→Outage then P(Outage|DB_Latency,HeavyTraffic,U) = P(Outage|DB_Latency,HeavyTraffic)

### Estimand : 2
Estimand name: iv
Estimand expression:
 ⎡                                                 -1⎤
 ⎢     d               ⎛     d                    ⎞  ⎥
E⎢────────────(Outage)⋅⎜────────────([DB_Latency])⎟  ⎥
 ⎣d[DB_Change]         ⎝d[DB_Change]              ⎠  ⎦
Estimand assumption 1, As-if-random: If U→→Outage then ¬(U →→{DB_Change})
Estimand assumption 2, Exclusion: If we remove {DB_Change}→{DB_Latency}, then ¬({DB_Change}→Outage)

### Estimand : 3
Estimand name: frontdoor
Estimand expression:
 ⎡                   d                                  ∂                      ↪
E⎢──────────────────────────────

In [13]:
# Step 3: Estimate
# Choose the second estimand: using IV
estimate = mod.estimate_effect(identified_estimand,
                                 method_name="backdoor.propensity_score_weighting", 
                                 test_significance=True,
                                 confidence_intervals=True)

print(estimate)

*** Causal Estimate ***

## Identified estimand
Estimand type: EstimandType.NONPARAMETRIC_ATE

### Estimand : 1
Estimand name: backdoor
Estimand expression:
      d                              
─────────────(E[Outage|HeavyTraffic])
d[DB_Latency]                        
Estimand assumption 1, Unconfoundedness: If U→{DB_Latency} and U→Outage then P(Outage|DB_Latency,HeavyTraffic,U) = P(Outage|DB_Latency,HeavyTraffic)

## Realized estimand
b: Outage~DB_Latency+HeavyTraffic
Target units: ate

## Estimate
Mean value: 0.35834243000752397
p-value: [0, 0.001]
95.0% confidence interval: (0.32775316125581067, 0.38418893312231983)



In [14]:
ref = mod.refute_estimate(identified_estimand, 
                            estimate, 
                            method_name="placebo_treatment_refuter", 
                            placebo_type="permute")